In [1]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData
from sqlalchemy.orm import mapper
from config import password

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

import pandas as pd


In [2]:
path = "cleaned_data/gender_race.csv"
pay_gap_by_gender_table = pd.read_csv(path, encoding="UTF-8")
pay_gap_by_gender_table.index.name = 'id'
pay_gap_by_gender_table.head()

,categoryindex,category,total_workers,median_weekly_earnings,year
id,,,,,
0,Total,Total,113272,860,2017
1,Total_Men,Men,62980,941,2017
2,Total_Women,Women,50291,770,2017
3,White_Total,White,87730,890,2017
4,White_Men,Men,50003,971,2017


In [3]:
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/gender_pay_gap')


In [4]:
#engine.table_names()
metadata = MetaData()
metadata.reflect(bind=engine)
print(metadata.tables)

immutabledict({'by_race': Table('by_race', MetaData(bind=None), Column('id', INTEGER(), table=<by_race>, primary_key=True, nullable=False), Column('household_data', VARCHAR(length=28), table=<by_race>), Column('total_workers_2017', INTEGER(), table=<by_race>), Column('total_workers_2018', INTEGER(), table=<by_race>), Column('median_weekly_earnings_2017', INTEGER(), table=<by_race>), Column('median_weekly_earnings_2018', INTEGER(), table=<by_race>), schema=None), 'pay_gap_by_race': Table('pay_gap_by_race', MetaData(bind=None), Column('id', INTEGER(), table=<pay_gap_by_race>, primary_key=True, nullable=False), Column('categoryindex', VARCHAR(length=50), table=<pay_gap_by_race>, nullable=False), Column('category', VARCHAR(length=50), table=<pay_gap_by_race>), Column('total_workers', INTEGER(), table=<pay_gap_by_race>), Column('median_weekly_earnings', INTEGER(), table=<pay_gap_by_race>), Column('year', INTEGER(), table=<pay_gap_by_race>), schema=None), 'pay_gap_by_education': Table('pay_g

In [5]:
pay_gap_by_gender_table.to_sql(name='pay_gap_by_race', con=engine, if_exists='append', index=True)


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pay_gap_by_race_pkey"
DETAIL:  Key (id)=(0) already exists.

[SQL: INSERT INTO pay_gap_by_race (id, categoryindex, category, total_workers, median_weekly_earnings, year) VALUES (%(id)s, %(categoryindex)s, %(category)s, %(total_workers)s, %(median_weekly_earnings)s, %(year)s)]
[parameters: ({'id': 0, 'categoryindex': 'Total', 'category': 'Total', 'total_workers': 113272, 'median_weekly_earnings': 860, 'year': 2017}, {'id': 1, 'categoryindex': 'Total_Men', 'category': 'Men', 'total_workers': 62980, 'median_weekly_earnings': 941, 'year': 2017}, {'id': 2, 'categoryindex': 'Total_Women', 'category': 'Women', 'total_workers': 50291, 'median_weekly_earnings': 770, 'year': 2017}, {'id': 3, 'categoryindex': 'White_Total', 'category': 'White', 'total_workers': 87730, 'median_weekly_earnings': 890, 'year': 2017}, {'id': 4, 'categoryindex': 'White_Men', 'category': 'Men', 'total_workers': 50003, 'medi

# Loading data for gender_pay_gap_byCoutry

In [6]:
global_data= pd.read_csv("cleaned_data/by_global.csv")
coords_data= pd.read_csv("cleaned_data/cords.csv")

In [7]:
global_data.head(5)

,Country,Sex,Occupation,Currency,Survey,Type_of_Source,Year_2016
0,Albania,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),General Directorate of Taxation,Insurance records,403
1,Albania,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),General Directorate of Taxation,Insurance records,357
2,Armenia,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Household Labour Force Survey,Labour force survey,249
3,Armenia,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Household Labour Force Survey,Labour force survey,166
4,Azerbaijan,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour-related establishment survey,Establishment survey,395


In [8]:
coords_data.head(5)


,Country,Capital,Latitude,Longitude
0,Afghanistan,Kabul,34°28'N,69°11'E
1,Albania,Tirane,41°18'N,19°49'E
2,Algeria,Algiers,36°42'N,03°08'E
3,American Samoa,Pago Pago,14°16'S,170°43'W
4,Andorra,Andorra la Vella,42°31'N,01°32'E


In [9]:
# Merge two dataframes using an inner join
global_coords_merged_table = pd.merge(coords_data, global_data ,on="Country")
global_coords_merged_table.index.name = 'id'

global_coords_merged_table.head(100)

,Country,Capital,Latitude,Longitude,Sex,Occupation,Currency,Survey,Type_of_Source,Year_2016
id,,,,,,,,,,
0,Albania,Tirane,41°18'N,19°49'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),General Directorate of Taxation,Insurance records,403
1,Albania,Tirane,41°18'N,19°49'E,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),General Directorate of Taxation,Insurance records,357
2,Armenia,Yerevan,40°10'N,44°31'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Household Labour Force Survey,Labour force survey,249
3,Armenia,Yerevan,40°10'N,44°31'E,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Household Labour Force Survey,Labour force survey,166
4,Azerbaijan,Baku,40°29'N,49°56'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour-related establishment survey,Establishment survey,395
5,Azerbaijan,Baku,40°29'N,49°56'E,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour-related establishment survey,Establishment survey,199
6,Bangladesh,Dhaka,23°43'N,90°26'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour Force Survey,Labour force survey,167
7,Bangladesh,Dhaka,23°43'N,90°26'E,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour Force Survey,Labour force survey,154
8,Belgium,Brussels,50°51'N,04°21'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Structure of Earnings Survey,Establishment survey,3634


In [10]:
global_coords_merged_table = global_coords_merged_table.drop(['Capital'], axis=1)


global_coords_merged_table = global_coords_merged_table.rename(columns={"id": "id", 
                            "Country": "country",
                           "Latitude": "latitude",
                           "Longitude": "longitude",
                           "Sex": "sex",
                           "Occupation": "occupation",
                           "Currency": "currency",
                           "Survey": "survey",
                           "Type_of_Source": "type_of_source",
                           "Year_2016": "year_2016_earnings"})



In [11]:
global_coords_merged_table.to_csv("Global_coords.csv")
global_coords_merged_table.head(10)

,country,latitude,longitude,sex,occupation,currency,survey,type_of_source,year_2016_earnings
id,,,,,,,,,
0,Albania,41°18'N,19°49'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),General Directorate of Taxation,Insurance records,403
1,Albania,41°18'N,19°49'E,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),General Directorate of Taxation,Insurance records,357
2,Armenia,40°10'N,44°31'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Household Labour Force Survey,Labour force survey,249
3,Armenia,40°10'N,44°31'E,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Household Labour Force Survey,Labour force survey,166
4,Azerbaijan,40°29'N,49°56'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour-related establishment survey,Establishment survey,395
5,Azerbaijan,40°29'N,49°56'E,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour-related establishment survey,Establishment survey,199
6,Bangladesh,23°43'N,90°26'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour Force Survey,Labour force survey,167
7,Bangladesh,23°43'N,90°26'E,Female (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Labour Force Survey,Labour force survey,154
8,Belgium,50°51'N,04°21'E,Male (Sex),Total (Occupation (ISCO-08)),U.S. dollars (Currency),Structure of Earnings Survey,Establishment survey,3634


In [12]:
global_coords_merged_table.to_sql(name='pay_gap_by_country', con=engine, if_exists='append', index=True)

# Loading data for byEducation

In [13]:
path_by_education = "cleaned_data/by_education_2016-2018.csv"
pay_gap_by_education = pd.read_csv(path_by_education, encoding="UTF-8")
pay_gap_by_education.index.name = 'id'
pay_gap_by_education.head()

,By education,Median weekly earnings in 2018,Median weekly earnings in 2017,Median weekly earnings in 2016,Gender
id,,,,,
0,"Men, 25 years and over",1026,996,969,Male
1,Less than a high school diploma,607,584,551,Male
2,"High school graduates, no college(3)",819,797,769,Male
3,Some college or associate degree,951,917,896,Male
4,"Some college, no degree",918,886,861,Male


In [14]:
pay_gap_by_education = pay_gap_by_education.rename(columns ={'By education': 'education_level',
                               'Median weekly earnings in 2018': 'median_weekly_earnings_in_2018',
                               'Median weekly earnings in 2017': 'median_weekly_earnings_in_2017',
                               'Median weekly earnings in 2016': 'median_weekly_earnings_in_2016',
                                                            'Gender':'gender'})
pay_gap_by_education.head()

,education_level,median_weekly_earnings_in_2018,median_weekly_earnings_in_2017,median_weekly_earnings_in_2016,gender
id,,,,,
0,"Men, 25 years and over",1026,996,969,Male
1,Less than a high school diploma,607,584,551,Male
2,"High school graduates, no college(3)",819,797,769,Male
3,Some college or associate degree,951,917,896,Male
4,"Some college, no degree",918,886,861,Male


In [15]:
pay_gap_by_education.dtypes

education_level                   object
median_weekly_earnings_in_2018     int64
median_weekly_earnings_in_2017     int64
median_weekly_earnings_in_2016     int64
gender                            object
dtype: object

In [16]:
 # Use pd.to_numeric() method to convert the datatype of the Amount column
pay_gap_by_education['median_weekly_earnings_in_2018'] = pd.to_numeric(pay_gap_by_education['median_weekly_earnings_in_2018'])

In [17]:
list(pay_gap_by_education.columns)

['education_level',
 'median_weekly_earnings_in_2018',
 'median_weekly_earnings_in_2017',
 'median_weekly_earnings_in_2016',
 'gender']

In [18]:
#engine.table_names()
metadata = MetaData()
metadata.reflect(bind=engine)
print(metadata.tables)

immutabledict({'by_race': Table('by_race', MetaData(bind=None), Column('id', INTEGER(), table=<by_race>, primary_key=True, nullable=False), Column('household_data', VARCHAR(length=28), table=<by_race>), Column('total_workers_2017', INTEGER(), table=<by_race>), Column('total_workers_2018', INTEGER(), table=<by_race>), Column('median_weekly_earnings_2017', INTEGER(), table=<by_race>), Column('median_weekly_earnings_2018', INTEGER(), table=<by_race>), schema=None), 'pay_gap_by_race': Table('pay_gap_by_race', MetaData(bind=None), Column('id', INTEGER(), table=<pay_gap_by_race>, primary_key=True, nullable=False), Column('categoryindex', VARCHAR(length=50), table=<pay_gap_by_race>, nullable=False), Column('category', VARCHAR(length=50), table=<pay_gap_by_race>), Column('total_workers', INTEGER(), table=<pay_gap_by_race>), Column('median_weekly_earnings', INTEGER(), table=<pay_gap_by_race>), Column('year', INTEGER(), table=<pay_gap_by_race>), schema=None), 'pay_gap_by_education': Table('pay_g

In [19]:
pay_gap_by_education.to_sql(name='pay_gap_by_education', con=engine, if_exists='append', index=True)